In [67]:
import pandas as pd
import numpy as np
import os

In [361]:
def load_Java_API_dependencies(file_name, sheet_name):
    return pd.read_excel(file_name,sheet_name=sheet_name, skiprows=2, usecols='B:WW',index_col=0,header=1 )

df_java_api = load_Java_API_dependencies('server-side.xlsm', 'Зависимости.JavaAPI')
df.head()

,ФП,Unnamed: 2,Unnamed: 3,Внешний шлюз,Unnamed: 5,Unnamed: 6,Stand-In,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246
NaN,NaN,Сервис,NaN,Проксирование запросов PL - Rest,Обновление конфигурации,Мониторинг состояния,Авторизация клиента в блоке,Запуской прямой репликации,Изменение ежима блока,Инициализация контекста ФП,...,SessionContext Удаление common-атрибута,SessionContext Установить код тербанка для сес...,SessionContext Получить код тербанка из сессии...,SessionContext Установить код ГОСБ для сессии ...,SessionContext Получить код ГОСБ из сессии канала,SessionContext Установить код ВСП для сессии к...,SessionContext Получить код ВСП из сессии канала,SessionContext Пометка текущей сессии invalidated,SessionContext Обновление секций сессии,SessionContext отложенная загрузка указанных с...
NaN,NaN,NaN,№ пп,1,2,3,4,5,6,7,...,234,235,236,237,238,239,240,241,242,243
1.0,Внешний шлюз,Проксирование запросов PL - Rest,1,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,Обновление конфигурации,2,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,Мониторинг состояния,3,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
def replace_multi(string, replace_list, replace_string=''):
    for elem in replace_list :
        if elem in string :
            string = string.replace(elem, replace_string)    
    return  string

In [311]:
# class Dependency:
    
#     def __init__(self, dep_name, dep_type):
#         self.name = dep_name
#         self.type = dep_type
        
#     def __str__(self):
#         return 'name: ' + self.name + '; type: ' + self.type

In [374]:
class Metadata:
    
    def __init__(self, file_name):
        self.meta = pd.read_excel(file_name, encoding='1251')

    def get(self, id_column, id_value, data_column):
        
        values = self.meta.loc[self.meta[id_column]==id_value][data_column].values
        if values.shape[0]>0:
            value = values[0]
            if value != value:  # NaN
                value = ''                
        else:
            value = ''
            
        return value

    def get_name(self, id_column_name, id_value):
        name = self.get(id_column_name, id_value, 'name')
        if name == '':
            print('for "{}"={} not found "name" in {}'.format(id_column_name, id_value, metadata_file))
        return name
    
#     def get_by_service_name(self, id_value, data_columns):
#         return self.get('service name', id_value, data_columns)

#     def get_by_client_lib(self, id_value, data_columns):
#         return self.get('client-lib name', id_value, data_columns)

#     def get_by_server_side(self, id_value, data_columns):
#         return self.get('server-side name', id_value, data_columns)
    
metadata_file = 'metadata.xlsx'    
meta = Metadata(metadata_file)    
meta.get( 'client-lib name', 'SSD', 'link')
meta.get( 'server-side name', 'Технологическое ядро', 'link')
meta.get( 'server-side name', 'Сессионные данные', ['client-lib name'])

array(['SSD'], dtype=object)

In [94]:
START_SUBSYSTEMS_COL = 3

def get_java_api_subsystems(df):
    subsystems = {}
    for i in range(START_SUBSYSTEMS_COL, len(df.columns)):
        column = df.columns[i]
        if not column.startswith('Unnamed:'):
            subsystem_name = column
            subsystems[subsystem_name] = i
    return subsystems

subsystems = get_java_api_subsystems(df_java_api)


In [369]:
def get_java_api_deps_list(df, subsystems):
    dep_list = {}

    keys = list(subsystems.keys())

    for i in range(len(keys)):

        key = keys[i]

        #calc subsystem columns
        start = subsystems[key]    
        if i < len(keys) - 1:
            end = subsystems[keys[i + 1]]
        else:
            end = df.shape[1]  # last column

        subsystem_columns = df.iloc[:,start:end]
        subsystem_deps = {}

        # now go through dependencies
        for j in range(len(keys)):
            dep_key = keys[j]

            # calc dependencies rows        
            dep_start = subsystems[dep_key] - 1 #rows shifted by -1
            if j < len(keys) - 1:
                dep_end = subsystems[keys[j + 1]] - 1
            else:
                dep_end = df.shape[0]  # last row

            dep_rows = subsystem_columns.iloc[dep_start:dep_end]
            calls_count = np.sum(np.sum(dep_rows))

            if  calls_count > 0:
                dep_name = meta.get_name('server-side name', dep_key)
                subsystem_deps[dep_name] = ['- серверная часть']                
        
        name = meta.get_name('server-side name', key)
        dep_list[name] = subsystem_deps
        
    return dep_list

In [388]:
def load_client_libs_dependenciec(file_name):
    df = pd.read_excel(file_name, encoding='1251', index_col=0)
    #df.head()
    # columns = ['Service']
    # columns.extend(list(df.iloc[:,0]))
    # columns.append('Comment')
    # print(columns)
    # df.columns= columns
    # df.head()
    df.columns = df.columns.map(lambda s: s.split('(')[0].strip())
    return df
    #df.head(30)

df_client_libs = load_client_libs_dependenciec('client-libs.xlsx')
df_client_libs.head()

,Сервис \ Переход на API,Справочники,Авторизация,SSD,СУП,Ошибки,Подсказки,Журнал,Аудит,Мониторинг,...,ufs-platform-httpclient,ufs-platform-workflow,ufs-platform-filestorage,ufs-platfrom-filetransfer,ТП,Фродмониторинг,Комплаенс,Подтверждение. Докат,Unnamed: 27,Unnamed: 28
№,,,,,,,,,,,,,,,,,,,,,
1.0,Справочники,NaN,X,X,+,X,X,+,X,X,...,Х,+,Х,Х,Х,Х,Х,Х,Х,Комментарий
2.0,Авторизация,X,NaN,+,+,X,X,+,X,X,...,X,+,X,X,X,Х,Х,Х,Х,Комментарий
3.0,SSD,X,X,NaN,X,X,X,+,X,X,...,X,+,X,X,X,X,Х,Х,Х,NaN
4.0,СУП,X,X,X,NaN,X,X,+,X,X,...,Х,Х,Х,X,X,X,Х,Х,Х,Комментарий
5.0,Ошибки,X,X,X,+,NaN,X,+,X,X,...,X,+,Х,X,X,X,Х,Х,Х,NaN


In [391]:
def get_client_libs_dependencies(df, deps):    
    
    df.fillna('X', inplace=True)
    
    for row in range(df.shape[1] - 1):
        
        name = df.columns[row + 1]
        if name.startswith('Unnamed:'): # comments column & etc.
            continue
            
        name = meta.get_name('client-lib name', name)        
        dep_dic = {}
        
        for col in range(1, df.shape[1]-1):
            
            cell_value = df.iloc[row, col]
            
            if cell_value not in ('X', 'Х'):
                
                dep_name = df.columns[col]
                dep_name = meta.get_name('client-lib name', dep_name)
        
                if name not in deps.keys():
                    deps[name] = {}

                if dep_name not in deps[name].keys():
                    deps[name][dep_name] = []

                deps[name][dep_name].append('- клиентский модуль')
            
    return deps

dependencies = get_java_api_deps_list(df_java_api, subsystems)
get_client_libs_dependencies(df_client_libs, dependencies)
#dependencies #['Прикладной мониторинг']

{'Внешний шлюз': {'Аудит': ['- серверная часть'],
  'Прикладной мониторинг': ['- серверная часть'],
  'Реестр Сервисов': ['- серверная часть']},
 'Stand-In': {'Аудит': ['- серверная часть'],
  'Справочники': ['- серверная часть'],
  'Управление параметрами': ['- серверная часть', '- клиентский модуль'],
  'ufs-platform-cache': ['- клиентский модуль'],
  'ufs-platform-workflow': ['- клиентский модуль']},
 'Авторизация': {'Аудит': ['- серверная часть'],
  'Журналирование': ['- серверная часть', '- клиентский модуль'],
  'Сессионные данные': ['- клиентский модуль'],
  'Управление параметрами': ['- клиентский модуль'],
  'ufs-platform-cache': ['- клиентский модуль'],
  'ufs-platform-workflow': ['- клиентский модуль']},
 'Аудит': {'Журналирование': ['- серверная часть', '- клиентский модуль']},
 'Журналирование': {'Авторизация': ['- серверная часть'],
  'Аудит': ['- серверная часть'],
  'Журналирование': ['- серверная часть'],
  'Сессионные данные': ['- клиентский модуль'],
  'Управление па

In [386]:
meta.meta['name']

0                                Справочники
1                                Авторизация
2                          Сессионные данные
3                     Управление параметрами
4                                     Ошибки
5                                  Подсказки
6                             Журналирование
7                                      Аудит
8                      Прикладной мониторинг
9                                   Stand-In
10              Защита от внутренних отказов
11                                    Печать
12                     Интеграционный модуль
13                          История операций
14                       Асинхронные задания
15                      Технологическое ядро
16                         ufs-platform-core
17                      ufs-platform-storage
18                        ufs-platform-cache
19                   ufs-platform-httpclient
20                     ufs-platform-workflow
21                  ufs-platform-filestorage
22        

In [135]:
#metadata.loc[metadata['service name']=='Справочники'][['client-lib name','number']]
#get_service_metadata(metadata, 'service name', 'Справочники', ['client-lib name','number'])

,client-lib name,number
2,SSD,3


In [377]:
SERVICE_LINK='link'
LINE_BREAK='&lt;br&gt;'

HEADER_HEIGHT = 90
SERVICE_X = 100  # service shape left
SERVICE_Y = 140  # service shape top
SHAPE_WIDTH = 300   # any shape width
SHAPE_HEIGHT = 110  # any shape width
X_MARGIN = 200 
Y_MARGIN = 50

DEP_X = SERVICE_X + SHAPE_WIDTH + X_MARGIN

def id_generator():    
    id_ = 3  # dependency ids starts from 3
    while True:
        yield str(id_)
        id_ += 1
get_id = id_generator()

def generate_dependencies_maps(template_file_name,
                               dependencies, 
                               meta):

    template = open(template_file_name,'r', encoding='utf-8').read()
    schema = template

    diagrams_folder = 'diagrams'
    if not os.path.exists(diagrams_folder):
        os.makedirs(diagrams_folder)

    for name in dependencies.keys():  # server_side_deps.keys():
        file_name = replace_multi(name, [':','\\','/'])
        with open(diagrams_folder + '\\' + name + '.xml','w', encoding='utf-8') as f:

            link = str(meta.get('name', name, SERVICE_LINK))
            if link != '':
                link = 'link="' + link + '"'

            schema = template
            schema = schema.replace('$$HEADER$$', 'Карта зависимостей "' + name + '"')
            schema = schema.replace('$$SERVICE$$', name)
            schema = schema.replace('$$SERVICE_LINK$$', link)

            splitted_schema = schema.split('$$DEP_BLOCK$$')
            schema = splitted_schema[0]
            dep_block = splitted_schema[1]
            dep_number = 0

            dep_figures = generate_dependencies(                
                    dependencies[name],
                    dep_block,
                    link)

            schema += dep_figures
            schema += splitted_schema[2]             
            f.write(schema)


def generate_dependencies(deps,
                          template,                    
                          link):    
    
    schema = ''
    dep_number = 0
    
    for dep_name in deps.keys():
        dep_y = SERVICE_Y + dep_number * (SHAPE_HEIGHT + Y_MARGIN)                
        
        dep_types = 'Что зависит:' + LINE_BREAK + LINE_BREAK.join(deps[dep_name])

        dep_block = template.replace('$$DEPENDENCY$$', dep_name)
        dep_block = dep_block.replace('$$DEP_LINK$$', link)
        dep_block = dep_block.replace('$$DEP_X$$', str(DEP_X))
        dep_block = dep_block.replace('$$DEP_Y$$', str(dep_y))

        dep_block = dep_block.replace('$$DEP_ID$$', next(get_id))
        dep_block = dep_block.replace('$$ARROW_ID$$', next(get_id))
        dep_block = dep_block.replace('$$DEP_TYPES_ID$$', next(get_id))

        dep_block = dep_block.replace('$$DEP_TYPES$$', dep_types)
        dep_block = dep_block.replace('$$DEP_TYPES_X$$', str(DEP_X+170))
        dep_block = dep_block.replace('$$DEP_TYPES_Y$$', str(dep_y+SHAPE_HEIGHT - 20))

        schema += dep_block
        dep_number += 1

    return schema
            
    # '$$SERVICE$$'
    # '$$SERVICE_LINK$$'
    # '$$DEPENDENCY$$'
    
    # '$$DEP_LINK$$'
    # '$$DEP_X$$'
    # '$$DEP_Y$$'
    # '$$DEP_ID$$'
    # '$$DEP_START$$'
    # '$$DEP_END$$'
    
    # '$$DEP_TYPES_ID$$'
    # '$$DEP_TYPES$$'de
    # '$$DEP_TYPES_X$$'
    # '$$DEP_TYPES_Y$$'
    
    # 'Java API&lt;br&gt;client jar&lt;br&gt;''

    #"/pages/viewpage.action?pageId=604506616"
generate_dependencies_maps('template.xml', dependencies, meta)

In [343]:
dependencies

{'Внешний шлюз': {'Аудит': ['Java API'],
  'Прикладной мониторинг': ['Java API'],
  'Реестр Сервисов': ['Java API']},
 'Stand-In': {'Аудит': ['Java API'],
  'Справочники': ['Java API'],
  'Управление параметрами': ['Java API', 'client jar', 'client jar'],
  'Технологическое ядро. Core': ['client jar'],
  'Технологическое ядро. Кэш': ['client jar'],
  'Технологическое ядро. Http-client': ['client jar'],
  'ufs-platform-core': ['client jar'],
  'ufs-platform-cache': ['client jar'],
  'ufs-platform-httpclient': ['client jar']},
 'Авторизация': {'Аудит': ['Java API'],
  'Журналирование': ['Java API', 'client jar', 'client jar'],
  'Сессионные данные': ['client jar', 'client jar'],
  'Управление параметрами': ['client jar', 'client jar'],
  'Технологическое ядро. Кэш': ['client jar'],
  'Технологическое ядро. Http-client': ['client jar'],
  'ufs-platform-cache': ['client jar'],
  'ufs-platform-httpclient': ['client jar']},
 'Аудит': {'Журналирование': ['Java API', 'client jar', 'client jar'

In [282]:
str(meta.get('server-side name', 'Управление параметрами', 'client-lib name'))


'СУП'